# --- Day 18: Lavaduct Lagoon ---

https://adventofcode.com/2023/day/18

## Parse the Input Data

In [1]:
from collections import defaultdict

In [2]:
def parse(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------
    dig_plan : dict
    """
    dig_plan = {}
    r, c = 0, 0

    rl = {"R" : 1, "L" : -1}
    ud = {"U" : -1, "D" : 1}

    with open(f'../inputs/{filename}.txt') as f:
        for line in f:
            d, n, color = line.strip().split()
            if d in "RL":
                for _ in range(1, int(n) + 1):
                    c += rl[d]
                    dig_plan[complex(r, c)] = color.strip("()")
            elif d in "UD":
                for _ in range(1, int(n) + 1):
                    r += ud[d]
                    dig_plan[complex(r, c)] = color.strip("()")

    return dig_plan

In [3]:
# Failed ray casting approach to the solution...
# def ray_casting():

#     answer = 0
#     for row in rows:
#         sorted_row = sorted(rows[row])
#         inside = True #|not len(sorted_row) % 2
#         row_count = 1
#         for i, _ in enumerate(sorted_row):
#             if i >= 1:
#                 if abs(sorted_row[i] - sorted_row[i-1]) == 1:
#                     row_count += 1
#                     # contig_count += 1
#                 else:
#                     row_count += inside * (abs(sorted_row[i] - sorted_row[i-1]))
#                     inside = not inside
#         answer += row_count

#         print(row_count, answer)
#     print(answer)

## Part 1
---

In [4]:
def shoelace(perimeter):
    """Calc area of a simple polygon using the shoelace formula.

    Parameters
    ---------
    perimeter : list
        Elements contain either complex(row, col) numbers or (row, col) tuples.
        Perimeter elements MUST be ordered in either a clockwise or counter-clockwise order.

    Returns
    -------
    area : float
        The area of a simple (non-overlapping) polygon.
    """
    if isinstance(perimeter[0], complex):
        temp = sum([(perimeter[i].real * perimeter[i+1].imag) - (perimeter[i].imag * perimeter[i+1].real) for i in range(len(perimeter) - 1)])
        # This last calc is zero if either start or end is (0, 0)
        temp += (perimeter[-1].real * perimeter[0].imag) - (perimeter[-1].imag * perimeter[0].real)

    else:
        temp = sum([(perimeter[i][0] * perimeter[i+1][1]) - (perimeter[i][1] * perimeter[i+1][0]) for i in range(len(perimeter) - 1)])
        print(temp)
        temp += (perimeter[-1][0] * perimeter[0][1]) - (perimeter[-1][1] * perimeter[0][0])

    area = temp / 2

    return area


In [5]:
def picks_theorem(perimeter):
    """Apply Pick's theorem to calculate the number of interior points of a simple polygon.

    First calculates the area of the simply polygon using the shoelace formula.
    Then given the number of points in the perimeter and the area of the perimeter, it calculates
    the number of interior points.

    Parameters
    ----------
    perimeter : list
        Elements contain either complex(row, col) numbers or (row, col) tuples
        perimeter (_type_): _description_

    Returns:
    num_interior_points : int
        The number of interior points in a simple polygon
    """
    area = shoelace(perimeter)
    num_perimeter_points = len(perimeter)
    num_interior_points = abs(area) - (num_perimeter_points / 2) + 1

    return int(num_interior_points)

In [6]:
def solve1(filename):
    dig_plan = parse(filename)
    perimeter = list(dig_plan.keys())
    num_interior_points = picks_theorem(perimeter)

    answer = int(num_interior_points + len(perimeter))

    return answer

### Run on Test Data

In [7]:
solve1("test_dig_plan") == 62

True

### Run on Input Data

In [8]:
solve1("dig_plan")

68115

## Part 2
---

In [9]:
def solve2(filename):
    r, c = 0, 0  # Start off at row 0, col 0
    perim_len = 0  # perimeter length
    temp = 0  # will hold intermediate calcs for shoelace formula

    dirs = {"0" : "R", "1" : "D", "2" : "L", "3" : "U"}

    instructions = []
    with open(f'../inputs/{filename}.txt') as f:
        for line in f:
            _, _, color = line.strip().split()
            n = int(color[2:-2], 16)
            d = dirs[color[-2:-1]]
            instructions.append((d, n))

    # print(instructions)

    for i, step in enumerate(instructions):
        d, n = step  # d = direction, n = number of steps
        perim_len += n

        if d == "R":
            c += n
            temp += (r * (n - 1))
            if i < len(instructions) - 1:
                next_d = instructions[i+1][0]
                if next_d == "U":
                    temp += (r * c) - ((r-1) * c)
                elif next_d == "D":
                    temp += (r * c) - ((r+1) * c)

        elif d == "L":
            c -= n
            temp += (r * (n - 1)) * -1
            if i < len(instructions) - 1:
                next_d = instructions[i+1][0]
                if next_d == "U":
                    temp += (r * c) - ((r-1) * c)
                elif next_d == "D":
                    temp += (r * c) - ((r+1) * c)

        elif d == "U":
            r -= n
            temp += (c * (n - 1))
            if i < len(instructions) - 1:
                next_d = instructions[i+1][0]
                if next_d == "L":
                    temp += (r * (c-1)) - (r * c)
                elif next_d == "R":
                    temp += (r * (c+1)) - (r * c)

        elif d == "D":
            r += n
            temp += (c * (n - 1)) * -1
            if i < len(instructions) - 1:
                next_d = instructions[i+1][0]
                if next_d == "L":
                    temp += (r * (c-1)) - (r * c)
                elif next_d == "R":
                    temp += (r * (c+1)) - (r * c)

    area = temp / 2

    num_interior_points = abs(area) - (perim_len / 2) + 1

    return int(num_interior_points) + perim_len

### Run on Test Data

In [10]:
solve2("test_dig_plan") == 952_408_144_115

True

### Run on Input Data

In [11]:
solve2("dig_plan")

71262565063800